# 유사성 분석을 테스트하기 위한 노트북입니다.

## 준비

쓸모없는 칼럼을 제거한 전용 데이터를 만듭니다.

이 작업은 엑셀이 더 편하므로 엑셀에서 합니다.

1. 특수 npc를 전부 날립니다. 특수 npc들은 속성이 너무 부족합니다.
2. `special` 열은 지웁니다.
3. `image_photo`, `image_house` 열도 지웁니다. `image_icon`은 혹시 모르니 남겨 두겠습니다.
4. `gender` 열을 지우고 `gender_asia`를 `gender`로 이름을 바꿉니다.
5. 마찬가지로 `personality_subtype`, `catchphrase`, `favorite_song`, `favorite_saying`, `tier`, `rank`도 사용자에게 질문할 수 없는 항목이므로 버립니다.
6. 나머지는 좀 더 생각이 필요하거나 코드로 해야 편한 작업들인 것 같습니다.
7. 빈 항목이 없도록 주의

In [20]:
import ast
import calendar
import datetime
from functools import partial

import numpy as np
import pandas as pd

In [3]:
chars = pd.read_excel(
  './datasets/characters-similarity.xlsx',
  sheet_name='Table1'
)

In [3]:
chars.info()
chars.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              390 non-null    object
 1   name_en         391 non-null    object
 2   name_ko         391 non-null    object
 3   image_icon      391 non-null    object
 4   species         391 non-null    object
 5   gender          391 non-null    object
 6   birthday        391 non-null    object
 7   birthday_month  391 non-null    int64 
 8   birthday_day    391 non-null    int64 
 9   personality     391 non-null    object
 10  hobby           391 non-null    object
 11  styles          391 non-null    object
 12  colors          391 non-null    object
dtypes: int64(2), object(11)
memory usage: 39.8+ KB


,id,name_en,name_ko,image_icon,species,gender,birthday,birthday_month,birthday_day,personality,hobby,styles,colors
0,admiral,Admiral,일섭,https://acnhcdn.com/latest/NpcIcon/brd06.png,Bird,Male,01-27,1,27,무뚝뚝,자연,['쿨'],"['검정색', '파랑색']"
1,agents,Agent S,2호,https://acnhcdn.com/latest/NpcIcon/squ05.png,Squirrel,Female,07-02,7,2,아이돌,운동,"['심플', '액티브']","['검정색', '파랑색']"
2,agnes,Agnes,아그네스,https://acnhcdn.com/latest/NpcIcon/pig17.png,Pig,Female,04-21,4,21,단순 활발,놀이,"['심플', '엘레강스']","['하양색', '핑크색']"
3,al,Al,우락,https://acnhcdn.com/latest/NpcIcon/gor08.png,Gorilla,Male,10-18,10,18,먹보,운동,['액티브'],"['하양색', '빨강색']"
4,alfonso,Alfonso,알베르트,https://acnhcdn.com/latest/NpcIcon/crd00.png,Alligator,Male,06-09,6,9,먹보,놀이,['심플'],"['빨강색', '파랑색']"


In [7]:
chars['styles'].iloc[0]

"['쿨']"

리스트들이 제대로 처리가 안 되어 있으니 변환합니다.

In [4]:
chars['styles'] = chars['styles'].apply(ast.literal_eval)
chars['colors'] = chars['colors'].apply(ast.literal_eval)
print(chars['styles'].head())
print(chars['colors'].head())

0           [쿨]
1     [심플, 액티브]
2    [심플, 엘레강스]
3         [액티브]
4          [심플]
Name: styles, dtype: object
0    [검정색, 파랑색]
1    [검정색, 파랑색]
2    [하양색, 핑크색]
3    [하양색, 빨강색]
4    [빨강색, 파랑색]
Name: colors, dtype: object


In [17]:
chars[['styles', 'colors']]

,styles,colors
0,[쿨],"[검정색, 파랑색]"
1,"[심플, 액티브]","[검정색, 파랑색]"
2,"[심플, 엘레강스]","[하양색, 핑크색]"
3,[액티브],"[하양색, 빨강색]"
4,[심플],"[빨강색, 파랑색]"
...,...,...
386,"[심플, 쿨]","[하양색, 회색]"
387,"[액티브, 쿨]","[초록색, 검정색]"
388,"[엘레강스, 쿨]","[오렌지색, 노랑색]"
389,"[고져스, 쿨]","[보라색, 회색]"


## 계획

유사성 평가에 사용할 속성은 아래의 5개입니다.
- birthday
- hobby
- personality
- style
- color

이 중 style과 color는 한 개일 수도 있고 두 개일 수도 있어서 약간 골치가 아픕니다.

### 비교 주체 고르기

overlap 비스무리한 분석은 항목이 여러 개인 카테고리도 같거나 다르거나로 취급하기 때문에
모든 캐릭터의 특성이 한 눈에 보이는 테이블은 만들기가 곤란합니다.
따라서 실험용으로 사용할 캐릭터가 하나 필요합니다.

그냥 맨 위에 있는 일섭이란 놈으로 합시다. (admiral)

### 대략적인 과정

1. 사용자에게 질문을 받습니다.
2. 5가지 속성 중 2가지로 먼저 필터링을 해서 연산 수를 줄입니다.
3. 남은 3속성은 3차원이므로 3차원 스캐터 그래프로 유사성을 보여줄 수 있습니다.
4. 캐릭터가 대략 400명이고, 2가지 필터링을 거치고 나면 많아봐야 100명 쯤 남을 것입니다.
5. 남은 캐릭터에 대해 벡터 크기를 계산하고 정렬합니다.
6. 벡터 크기는 차이입니다. 앞에 오는 캐릭터가 유사한 캐릭터, 뒤에 오면 다른 것입니다.

### 어떤 속성으로 먼저 걸러야 하는가?

그건 좀 생각을 해 봐야 합니다.

기본적으로는 한번에 최대한 많이, 그리고 균일하게 걸러낼 수 있는 속성이 좋을 것입니다.


In [21]:
chars.groupby('hobby').count()['id']

hobby
교육    64
놀이    65
운동    66
음악    64
자연    65
패션    66
Name: id, dtype: int64

In [22]:
chars.groupby('personality').count()['id']

personality
느끼함      34
단순 활발    24
먹보       60
무뚝뚝      55
성숙함      55
아이돌      49
운동광      55
친절함      58
Name: id, dtype: int64

In [23]:
chars.groupby('birthday').count()['id']

birthday
01-01    1
01-02    1
01-03    1
01-04    1
01-05    1
        ..
12-27    1
12-28    1
12-29    2
12-30    1
12-31    1
Name: id, Length: 361, dtype: int64

필터를 두 번만 하면 100명이 아니라 10명도 안 남을 것 같습니다.

윤년이 생일인 녀석(몽셰르)이 있으니 1년은 366일인데 위에선 361개밖에 없는 걸 보니 빠진 일이 5개가 있는 모양입니다. 그래도 상관 없습니다.

## 대안

앞의 방법은 속성 중 2가지에 대한 평가가 완전히 옳다고 치고 걸러내는 것입니다.

하지만 속성 한번에 대략 1/7이 걸러지므로 두 번을 하고 나면 평균적으로 비교할 캐릭터는 10명 남짓밖에 남지 않을 것입니다. 아마 상당히 심심한 그래프가 될 겁니다.

대안은 5차원을 전부 사용하고 대신 3차원 스캐터 그래프에 `hue`, `size` 값에 변화를 주어 차원 2개를 어거지로 만들어 내는 것입니다.

단점:
  - 사람에게 보이는 가까운 거리와 계산 결과 가까운 거리가 다를 수 있습니다.
  - 따라서 보여주니까 뭔가 멋지긴 하지만 사실상 그래프로 유의미한 해석을 하는 것은 힘들 것입니다.

따라서, 만약 저렇게 한다면 `hue`와 `size`로 나타내는 속성은 좀 덜 중요한 것을 넣고 비중을 조절한다든지 해야 할 것입니다.

또한, 만약 위의 방법으로 할 것에 대비해 5속성 모두에 대해 유사성을 평가하는 논리를 결정해 두어야 합니다.


## 평가 방법

모든 방법은 자바스크립트와 파이썬 모두에서 구현 가능해야 합니다.

### `birthday`

일 수로 바꾸어 날짜 차이를 계산합니다.
주의사항은 생일이 2월 29일인 녀석이 있습니다.

```
diff = abs(yday2 - yday1) % 183
```


In [5]:
def diff_dob(yday1, yday2):
  diff = yday1 - yday2
  if abs(diff) < 182.5:
    return abs(diff)
  else:
    return 182.5*2 - abs(diff)

In [6]:
print(diff_dob(1, 365))

assert diff_dob(1, 365) == 1
assert diff_dob(364, 1) == 2
assert diff_dob(31, 59) == 28
assert diff_dob(59, 31) == 28

1.0


In [46]:
# calendar.isleap(datetime.date.today().year)

# today = datetime.date.today()
# today.timetuple().tm_yday

day = datetime.date(2022, 1, 1)
day.timetuple().tm_yday

1

### `hobby`, `personality`

단순하게 같으면 0, 다르면 1입니다.

### `colors`, `styles`

두 캐릭터의 교집합을 구해서 길이를 보고 2면 1, 1이면 0.5, 0이면 0입니다.

## 일섭에게 적용해보기

일섭을 기준으로 다른 모든 캐릭터와의 비교를 마치면 최종 유사도 테이블의 한 행이 완성됩니다.

In [53]:
chars.head()

,id,name_en,name_ko,image_icon,species,gender,birthday,birthday_month,birthday_day,personality,hobby,styles,colors
0,admiral,Admiral,일섭,https://acnhcdn.com/latest/NpcIcon/brd06.png,Bird,Male,01-27,1,27,무뚝뚝,자연,[쿨],"[검정색, 파랑색]"
1,agents,Agent S,2호,https://acnhcdn.com/latest/NpcIcon/squ05.png,Squirrel,Female,07-02,7,2,아이돌,운동,"[심플, 액티브]","[검정색, 파랑색]"
2,agnes,Agnes,아그네스,https://acnhcdn.com/latest/NpcIcon/pig17.png,Pig,Female,04-21,4,21,단순 활발,놀이,"[심플, 엘레강스]","[하양색, 핑크색]"
3,al,Al,우락,https://acnhcdn.com/latest/NpcIcon/gor08.png,Gorilla,Male,10-18,10,18,먹보,운동,[액티브],"[하양색, 빨강색]"
4,alfonso,Alfonso,알베르트,https://acnhcdn.com/latest/NpcIcon/crd00.png,Alligator,Male,06-09,6,9,먹보,놀이,[심플],"[빨강색, 파랑색]"


`species`, `gender`, `birthday_x` 열은 버려도 될 것 같습니다.

In [55]:
chars = chars.drop(columns=['species', 'gender'])
chars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              390 non-null    object
 1   name_en         391 non-null    object
 2   name_ko         391 non-null    object
 3   image_icon      391 non-null    object
 4   birthday        391 non-null    object
 5   birthday_month  391 non-null    int64 
 6   birthday_day    391 non-null    int64 
 7   personality     391 non-null    object
 8   hobby           391 non-null    object
 9   styles          391 non-null    object
 10  colors          391 non-null    object
dtypes: int64(2), object(9)
memory usage: 33.7+ KB


In [7]:
# sims = pd.DataFrame(columns=chars['id'])
vectors = pd.DataFrame(
  columns=['id', 'birthday', 'hobby', 'personality', 'styles', 'colors']
)
vectors.head()

,id,birthday,hobby,personality,styles,colors


In [19]:
REF = chars.iloc[0]
print(REF)

id                                                     admiral
name_en                                                Admiral
name_ko                                                     일섭
image_icon        https://acnhcdn.com/latest/NpcIcon/brd06.png
species                                                   Bird
gender                                                    Male
birthday                                                 01-27
birthday_month                                               1
birthday_day                                                27
personality                                                무뚝뚝
hobby                                                       자연
styles                                                     [쿨]
colors                                              [검정색, 파랑색]
Name: 0, dtype: object


In [21]:
# for row in chars.itertuples(index=False, name='char'):
  # print(row['id'])
TODAY_YEAR = datetime.date.today().year
TODAY_LEAP = calendar.isleap(TODAY_YEAR)
DAY_MOD = 183.0 if TODAY_LEAP else 182.5

def compare_simple(a, b):
  return 0.0 if a == b else 1.0

def compare_set(a, b):
  i = set(a) & set(b)
  return 1.0 - len(i)/2.0

def to_yday(m: int, d: int) -> int:
  datetime.date(TODAY_YEAR, m, d).timetuple().tm_yday
  
def compare_yday(a: tuple, b: tuple):
  a = to_yday(*a)
  b = to_yday(*b)
  delta = abs(a - b)
  if delta < DAY_MOD:
    result = abs(delta)
  else:
    result = DAY_MOD*2 - delta
  return result / DAY_MOD / 2

compare_hobby = partial(compare_simple, REF.hobby)
compare_personality = partial(compare_simple, REF.personality)
compare_colors = partial(compare_set, REF.colors)
compare_styles = partial(compare_set, REF.styles)
compare_birthday = partial(compare_yday, (REF.birthday_month, REF.birthday_day))


이제 한번 돌려봅니다.

하지만 그전에 df 연습;

In [17]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})
df.loc[len(df)] = [4, 7, 10]
df

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9
3,4,7,10
